WICHTIG: ich brauche die zeit garnicht zu korrigieren, da ich komplett über die frames gehen kann. auch für die sync mit den pose daten.

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import h5py

### inspection

In [ ]:
# import mat files for A1
file_paths = glob.glob("../data/A1/*.mat")
file_paths.sort()
file_paths

In [ ]:
# get example file path
example_file_path = file_paths[0]
example_file_path

In [ ]:
# load .mat file
data = h5py.File(example_file_path, 'r')
data.keys()

Channel 3: TTLs an die Kamera\
Channel 4: Sound trigger\
Channel 31: Keyboard

#### keyboard times

In [ ]:
ch31 = data["Ch31"]
ch31.keys()

In [ ]:
np.array(ch31["codes"]).flatten()

In [ ]:
np.array(ch31["times"]).flatten()

In [ ]:
keyboard_trigger_times = np.array(ch31["times"]).flatten()[1:-1]
keyboard_trigger_times

In [ ]:
start_time = np.array(ch31["times"]).flatten()[0]
start_time

In [ ]:
keyboard_trigger_times_cut = keyboard_trigger_times - start_time
keyboard_trigger_times_cut

#### TTL trigger times

In [ ]:
ch3 = data["Ch3"]
ch3.keys()

In [ ]:
ttl = np.array(ch3["values"]).flatten()
ttl_times = np.array(ch3["times"]).flatten()
ttl.shape, ttl_times.shape

In [ ]:
ttl, ttl_times

cut to start with keyboard press

In [ ]:
ttl_cut = ttl[ttl_times > start_time]
ttl_times = ttl_times - ttl_times[0]
ttl_times_cut = ttl_times[ttl_times > start_time] - start_time
ttl_times_cut = ttl_times_cut - ttl_times_cut[0]
ttl_times_cut

In [ ]:
# count ttl triggers
ttl_triggers_idx = np.where(np.diff(ttl_cut) > 2)[0]
len(ttl_triggers_idx)

#### sound stimulus

In [ ]:
ch4 = data["Ch4"]
ch4.keys()

In [ ]:
sound = np.array(ch4["values"]).flatten()
sound_times = np.array(ch4["times"]).flatten()
sound.shape, sound_times.shape

In [ ]:
sound, sound_times

cut to start with keyboard press

In [ ]:
sound_cut = sound[sound_times > start_time]
sound_times = sound_times - sound_times[0]
sound_times_cut = sound_times[sound_times > start_time] - start_time
sound_times_cut = sound_times_cut - sound_times_cut[0]
sound_times_cut

plot all channels

In [ ]:
# plot ttl and sound signal in two separte windows below each other
fig, axs = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
axs[0].plot(ttl_times_cut, ttl_cut)
axs[0].set_title("TTL Signal")
axs[0].set_ylabel("TTL Value")
axs[0].set_xlim(sound_times_cut[0], sound_times_cut[-1] * 1.01)
axs[1].plot(sound_times_cut, sound_cut, color='orange')
axs[1].set_title("Sound Signal")
axs[1].set_ylabel("Sound Value")
axs[1].set_xlabel("Time (s)")
# plot keyboard triggers as vertical lines
for kt in keyboard_trigger_times_cut:
    axs[1].axvline(x=kt, color='red', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

look at minor difference


In [ ]:
ttl_times_cut

In [ ]:
sound_times_cut

In [ ]:
sound_cut.shape, ttl_cut.shape

compare to sound trigger times

In [ ]:
keyboard_trigger_times_cut

In [ ]:
# inpect 400 ms after every keyboard trigger for onset of sound trigger

sound_onset_times = np.empty_like(keyboard_trigger_times_cut)
for i, kt in enumerate(keyboard_trigger_times_cut):

    # get time window of sound signal 400 ms after keyboard trigger
    sound_window = sound_cut[(sound_times_cut > kt) & (sound_times_cut < kt + 0.4)]
    sound_times_window = sound_times_cut[(sound_times_cut > kt) & (sound_times_cut < kt + 0.4)]
    
    sound_trigger_onset = np.where(np.diff(sound_window) > 2)[0]
    if len(sound_trigger_onset) > 0:
        sound_onset_times[i] = sound_times_window[sound_trigger_onset[0] + 1]
    else:
        sound_onset_times[i] = np.nan  # no sound trigger found
        print(f"No sound trigger found for keyboard trigger at {kt:.3f} s")

In [ ]:
no_trials = len(sound_onset_times)
no_trials

sound onset times as start points of trials, each trial has 1s pretrial and 3s trial

get time / frame association: from index of sound onset, get index of first frame triggered after sound onset, get 30 frames before and 90 frames after this.

In [ ]:
ttl_cut

In [ ]:
ttl_times_cut

In [ ]:
sound_onset_times

### video and ttl syncronization

In [ ]:
import cv2
cap = cv2.VideoCapture("../data/A1/FH0002 25-11-14 14-37-02.mp4")

test if for a video it has the same number of frames as ttl triggers were sent, probably not, as some frames are dropped

In [ ]:
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total number of frames: {frame_count}")

In [ ]:
len(ttl_triggers_idx)

3 frames dropped

### routine to remove ahead time from data for 1 animal:
1 - ttl trigger times\
2 - sound stimulus times

not from video frames/pose data, since they automatically start at first frame sent

testing: compare frame number of video to frame number from ttl triggers

In [ ]:
# spike2 key codes
START_KEY = "C"
END_KEY = "A"


animal_path  = "../data/A1/"
animal_path = os.path.join(animal_path, "*.mat")
animal_path = os.path.abspath(animal_path)
animal_path

In [ ]:
for file in glob.glob(animal_path):
    data = h5py.File(file, "r")
    ch31 = data["Ch31"]
    codes = np.array(ch31["codes"]).flatten()
    times = np.array(ch31["times"]).flatten()

    # get start and end times of video recording from channel 31
    start_idx = np.where(codes == ord(START_KEY))[0][0]  # should be always zero
    assert start_idx == 0, "Start key not found at the beginning of the file"

    try:
        end_idx = np.where(codes == ord(END_KEY))[0][0]
    except IndexError:
        raise ValueError(f"End key missing in file {file}")

    start_time = times[start_idx]
    end_time = times[end_idx]

    # let recording time start at 0

    # TTL CHANNEL
    ch3 = data["Ch3"]
    ttl = np.array(ch3["values"]).flatten()
    ttl_times = np.array(ch3["times"]).flatten()
    ttl_times = ttl_times - ttl_times[0]  # set start time to 0

    ttl_cut = ttl[(ttl_times > start_time) & (ttl_times < end_time)]
    ttl_times_cut = ttl_times[(ttl_times > start_time) & (ttl_times < end_time)]
    ttl_times_cut = ttl_times_cut - start_time  # set start time beginning of key press

    # SOUND CHANNEL
    ch4 = data["Ch4"]
    sound = np.array(ch4["values"]).flatten()
    sound_times = np.array(ch4["times"]).flatten()
    sound_times = sound_times - sound_times[0]  # set start time to 0

    sound_cut = sound[(sound_times > start_time) & (sound_times < end_time)]
    sound_times_cut = sound_times[(sound_times > start_time) & (sound_times < end_time)]
    sound_times_cut = sound_times_cut - start_time  # set start time to 0

    # get stimulus trigger times
    ttl_triggers_idx = np.where(np.diff(ttl_cut) > 2)[0]
    n_triggers = len(ttl_triggers_idx)